In [1]:
import sqlite3
from flask import Flask, render_template, request, redirect, url_for, session, flash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'

@app.route('/')
def index():
    return render_template('login.html')

@app.route('/home')
def home():
    if 'username' in session:
        return render_template('home.html')
    else :
        return render_template('login.html')



In [2]:

@app.route('/myBook', methods=['GET', 'POST'])
def myBook():
    status = request.args.get('status', '')
    username = session.get('username', '')
    try:
        books = []
        with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            if not status:
                cur.execute("""
                            SELECT 
                                T1.title,
                                T2.name,
                                T3.avr_count,
                                T1.book_id,
                                T4.status,
                                T1.book_id
                            FROM
                                book AS T1 
                                JOIN author AS T2 ON T1.author_id = T2.author_id 
                                JOIN book_rating AS T3 ON T3.book_id = T1.book_id 
                                JOIN user_book AS T4 ON T4.book_id = T1.book_id
                            WHERE T4.username = ?
                            """, (username,))
            else:
                statusName = "%%"
                if status == '1':
                    statusName = 'read'
                if status == '2':
                    statusName = "reading"
                if status == '3':
                    statusName = 'to-read'

                cur.execute("""
                            SELECT 
                                T1.title,
                                T2.name,
                                T3.avr_count,
                                T1.book_id,
                                T4.status,
                                T1.book_id
                            from
                                book as T1 
                                JOIN author as T2 ON T1.author_id = T2.author_id 
                                JOIN book_rating as T3 ON T3.book_id = T1.book_id 
                                JOIN user_book as T4 ON T4.book_id = T1.book_id
                            WHERE T4.username = ? AND T4.status = ?
                            """, (username, statusName))
            books = cur.fetchall()
            if not books:
                msg = "Record successfully added to database"
            else:
                flash('Login successful!', 'success')
                return render_template('myBook.html', books=books)
            con.commit()
    except sqlite3.Error as e:
        con.rollback()
        print(e)
        msg = "Error in the Search"
        return render_template('myBook.html')
    finally:
        con.close()

    return render_template('myBook.html', status=1)

In [3]:

@app.route('/bookDescription', methods=['GET', 'POST'])
def bookDescription():
    book = []
    book_id = request.args.get('book_id', '')

    if request.method == 'POST':
        try:
            with sqlite3.connect('database.db') as con:
                book_id = request.form.get('book_id')
                username = session['username']
                status = 'to-read'
                print(book_id)
                cur = con.cursor()
                cur.execute("""
                                INSERT INTO user_book (book_id, username, status)
                                VALUES (?, ?, ?)
                            """, (book_id, username, status))
                con.commit()

                cur.execute(""" SELECT 
                                T1.title as title, 
                                T2.name as name, 
                                T3.avr_count, 
                                T3.review_count,
                                T3.rating_count,
                                T1.date_published,
                                T1.description,
                                T1.book_id
                            from 
                                book as T1 JOIN author as T2 ON T1.author_id = T2.author_id 
                                JOIN book_rating as T3 ON T3.book_id = T1.book_id 
                                JOIN book_info AS T4 ON T1.book_id = T4.book_id 
                            WHERE T1.book_id = '""" + book_id + "'")
                book = cur.fetchall()
                if not book:
                    msg = ""
                    print(msg)
                else:
                    flash('Login successful!', 'success')
                    print("update")
                    return render_template('bookDescription.html', book=book, book_id=book_id)

                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the Search"
            return render_template('bookDescription.html', msg=msg)
        finally:
            con.close()
    if request.method == 'GET':
        try:
            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                cur.execute(""" SELECT 
                                T1.title as title, 
                                T2.name as name, 
                                T3.avr_count, 
                                T3.review_count,
                                T3.rating_count,
                                T1.date_published,
                                T1.description
                            from 
                                book as T1 JOIN author as T2 ON T1.author_id = T2.author_id 
                                JOIN book_rating as T3 ON T3.book_id = T1.book_id 
                                JOIN book_info AS T4 ON T1.book_id = T4.book_id 
                            WHERE T1.book_id = '""" + book_id + "'")
                book = cur.fetchall()

                if not book:
                    msg = "Record successfully added to database"
                else:
                    flash('Login successful!', 'success')
                    return render_template('bookDescription.html', book=book, book_id=book_id)
                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the Search"
            return render_template('bookDescription.html')
        finally:
            con.close()
    return render_template('bookDescription.html', book=book, book_id=book_id)

In [4]:
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    username = session['username']
    try:
        with sqlite3.connect('database.db') as con:
            con.create_function("extract_year", 1, lambda x: int(x[:4]) if x else None)
            cur = con.cursor()
            years = ['2018', '2019', '2020', '2021', '2022', '2023']
            result_array = []

            for year in years:
                year_pattern = '%' + year + '%'
                cur.execute("""
                    SELECT COUNT(*) 
                    FROM user_book
                    WHERE status = 'read'
                        AND username = ?
                        AND CAST(substr(finish_date, 1, 4) AS INTEGER) = ?
                """, (username, int(year)))

                count = cur.fetchone()[0]
                result_array.append(count)

            # chart bar
            status = ['read', 'reading', 'to-read']
            result_bar_array = []

            for sts in status:
                cur.execute("""
                    SELECT COUNT(*) 
                    FROM user_book
                    WHERE status = ? AND username = ?
                """, (sts, username))

                count = cur.fetchone()[0]
                result_bar_array.append(count)

            chart_bar_data = {
                'labels': status,
                'values': result_bar_array
            }

            chart_line_data = {
                'labels': years,
                'data': result_array,
                'label': "the number of book 'read'"
                }

            print(result_array)

    except sqlite3.Error as e:
        con.rollback()
        print(e)
    else:
        print(chart_line_data)

    if request.method == 'POST':
        try:
            user=[]    
            Username = request.form.get('Username')
            FullName = request.form.get('FullName')
            Email = request.form.get('Email')
            Password = request.form.get('Password')

            
            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                username = session['username']
                cur.execute("UPDATE user SET username=?, fullname=?, email=?, password=?  WHERE username= '" + Username +"'", (Username, FullName, Email, Password))
                cur.execute("SELECT username,fullname,email,password FROM user where username = '" + Username + "'")
                user = cur.fetchall()
                if not user:
                    msg = "Record successfully added to database"
                else :
                    flash('Login successful!', 'success')
                    return render_template('profile.html',chart_data=chart_line_data,user = user,chart_bar_data=chart_bar_data)
                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the EditProfile"
            return render_template('profile.html',msg=msg)
        finally:
            con.close()
    try:        
        user=[]  
        
        with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            username = session['username']
            print(username)
            cur.execute("SELECT username,fullname,email,password FROM user where username = '" + username + "'")
            user = cur.fetchall()

            # 
            
            if not user:
                msg = "Record successfully added to database"
            else :
                flash('Login successful!', 'success')
                return render_template('profile.html',chart_data=chart_line_data,user = user,chart_bar_data=chart_bar_data)
            con.commit()
    except sqlite3.Error as e:
        con.rollback()
        print(e)
        msg = "Error in the EditProfile"
        return render_template('profile.html')
    finally:
        con.close()

    return render_template('profile.html',chart_data=chart_line_data,user = user,chart_bar_data=chart_bar_data)

In [5]:

@app.route('/search', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        try:
            books = []
            searchInput = request.form.get('searchInput')

            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                search_pattern = '%' + searchInput + '%'

                cur.execute("""
                            SELECT T1.title as title, T2.name as name, T3.avr_count, T1.book_id as year 
                            FROM book as T1 
                            JOIN author as T2 ON T1.author_id = T2.author_id 
                            JOIN book_rating as T3 ON T3.book_id = T1.book_id 
                            WHERE T2.name LIKE ? OR T1.title LIKE ? 
                            """, (search_pattern, search_pattern))

                books = cur.fetchall()

                if not books:
                    msg = "No matching records found."
                else:
                    flash('Search successful!', 'success')
                    return render_template('search.html', books=books)

                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the search."
            return render_template('search.html', msg=msg)
        finally:
            con.close()

    return render_template('search.html')

In [6]:
@app.route('/myNote', methods=['GET', 'POST'])
def myNote():
    book_id = request.args.get('book_id', '')
    if request.method == 'POST':
        Status = request.form.get('Status')
        StartDate = request.form.get('StartDate')
        FinishDate = request.form.get('FinishDate')
        Notes = request.form.get('Notes')
        try:
            
            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                cur.execute("""
                            UPDATE user_book 
                            SET 
                                status=?, 
                                start_date=?, 
                                finish_date=?, 
                                notes=?  
                            WHERE book_id= '""" + book_id +"'", 
                            (Status, StartDate, FinishDate, Notes))

                cur.execute("""
                            SELECT
                                T1.title,
                                T3.status,
                                T2.name,
                                T3.start_date,
                                T3.finish_date,
                                T3.notes
                            from
                                book as T1 
                                JOIN author as T2 ON T1.author_id = T2.author_id 
                                JOIN user_book as T3 ON T3.book_id = T1.book_id 
                                JOIN book_info as T4 on T1.book_id = T4.book_id 
                            WHERE T1.book_id = '""" + book_id + "'")
                book = cur.fetchall()

                if not book:
                    msg = "Record successfully added to database"
                else :
                    flash('Login successful!', 'success')
                    return render_template('myNote.html',book = book,book_id = book_id)
                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the Search"
            return render_template('bookDescription.html',msg=msg)
        finally:
            con.close()
            # Send the transaction message to result.html
            # return render_template('login.html',msg=msg)
    if request.method == 'GET':
        try:
            
            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                cur.execute("""
                            SELECT
                                T1.title,
                                T3.status,
                                T2.name,
                                T3.start_date,
                                T3.finish_date,
                                T3.notes
                            from
                                book as T1 
                                JOIN author as T2 ON T1.author_id = T2.author_id 
                                JOIN user_book as T3 ON T3.book_id = T1.book_id 
                                JOIN book_info as T4 on T1.book_id = T4.book_id 
                            WHERE T1.book_id = '""" + book_id + "'")
                book = cur.fetchall()

                if not book:
                    msg = "Record successfully added to database"
                else :
                    flash('Login successful!', 'success')
                    return render_template('myNote.html',book = book,book_id = book_id)
                con.commit()
        except sqlite3.Error as e:
            con.rollback()
            print(e)
            msg = "Error in the Search"
            return render_template('myNote.html',msg=msg)
        finally:
            con.close()
            # Send the transaction message to result.html
            # return render_template('login.html',msg=msg)
    return render_template('myNote.html')

In [7]:
@app.route('/signUp', methods=['GET', 'POST'])
def signUp():
    if request.method == 'POST':
        username = request.form['username']
        fullname = request.form['fullname']
        email = request.form['email']
        password = request.form['password']
        re_password = request.form['re_password']

        # Check if passwords match
        if password != re_password:
            return render_template('signUp.html', error='Passwords do not match')

        with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            cur.execute('SELECT * FROM user WHERE username = ?', (username,))
            existing_user = cur.fetchone()

        if existing_user:
            return render_template('signUp.html', error='Username already exists. Please choose a different one.')


        with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            cur.execute('SELECT * FROM user WHERE email = ?', (email,))
            existing_email = cur.fetchone()

        if existing_email:
            return render_template('signUp.html', error='Email already exists. Please choose a different one.')


        with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            cur.execute('INSERT INTO user (username, fullname, email, password) VALUES (?, ?, ?, ?)', (username, fullname, email, password))
            con.commit()

            return redirect(url_for('login'))

    return render_template('signup.html')

In [8]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    msg = ""
    if request.method == 'POST':
        try:
            username = request.form.get('username')
            password = request.form.get('password')
            with sqlite3.connect('database.db') as con:
                cur = con.cursor()
                cur.execute("SELECT username, password from user where username = ? AND password = ?",
                            (username, password))
                user = cur.fetchall()
                if not user:
                    msg = "Wrong password or username"
                else :
                    session['username'] = username
                    return redirect(url_for('home'))
                con.commit()
        except:
            con.rollback()
            msg = "Error in the Login"
            return render_template('login.html',msg=msg)
        finally:
            con.close()
    return render_template('login.html',msg=msg)

In [9]:
@app.route('/logout')
def logout():
    session.pop('username', None)
    flash('You have been logged out.', 'info')
    msg=""
    return render_template('login.html',msg=msg)
# end control login


In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 8080, app, use_reloader=False, use_debugger=False)

 * Running on http://localhost:8080
Press CTRL+C to quit
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/animate/animate.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/fonts/iconic/css/material-design-iconic-font.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/css-hamburgers/hamburgers.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/select2/select2.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/daterangepicker/daterangepicker.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09:11:15] "GET /static/vendor/animsition/css/animsition.min.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Feb/2024 09: